In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("transfusion_5%.csv")

In [3]:
df

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.0,50.0,12500.0,98.0,1.0
1,NaN,13.0,3250.0,NaN,1.0
2,1.0,16.0,4000.0,35.0,1.0
3,2.0,NaN,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0
...,...,...,...,...,...
743,NaN,2.0,500.0,38.0,0.0
744,21.0,2.0,500.0,52.0,0.0
745,23.0,NaN,750.0,62.0,0.0
746,39.0,1.0,250.0,39.0,0.0


In [4]:
df.dtypes

Recency (months)                              float64
Frequency (times)                             float64
Monetary (c.c. blood)                         float64
Time (months)                                 float64
whether he/she donated blood in March 2007    float64
dtype: object

In [5]:
df.isnull().sum()

Recency (months)                              46
Frequency (times)                             46
Monetary (c.c. blood)                         55
Time (months)                                 40
whether he/she donated blood in March 2007     0
dtype: int64

In [6]:
df.isnull().sum().sum()

187

In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Recency (months)                              6.149733
Frequency (times)                             6.149733
Monetary (c.c. blood)                         7.352941
Time (months)                                 5.347594
whether he/she donated blood in March 2007    0.000000
dtype: float64

In [8]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 3740
nilai hilang : 187
persentase : 5.0


In [9]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [10]:
from collections import Counter

def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count <= 2 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None    

def Final(df,col,k):
    terdekat = []
    mean = 0
    minimum = None
    maximum = None
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        if (minimum is None or maximum is None):
            minimum = y
            maximum = y
        elif(y<minimum):
            minimum = y
        elif(y>maximum):
            maximum = y
        mean += y
        euc_val.append(y)
    mean /= len(terdekat)
    euc_val.sort()
    print(euc_val)
    
    fuzzy = hitung(euc_val,5)
    print("min : ",minimum)
    print("max : ",maximum)
    print("mean : ",mean)
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,fuzzy_triangular(y,minimum,mean,maximum)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    
    if tertimbang == 0:
        tertimbang = dummy.iloc[index-1, col]
        
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [11]:
def fuzzy_triangular(val, minimum, mean, maximum):
   if val <= minimum or val >= maximum: 
        return 0.0
   elif minimum < val <= mean:
        return float((val - minimum)/(mean - minimum)) 
   elif mean < val < maximum:
        return float((maximum - val)/(maximum - mean))

In [12]:
data = df.iloc[:, :-1]
df2 = df.copy()
dummy = df2.fillna(0)
dummy

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.0,50.0,12500.0,98.0,1.0
1,0.0,13.0,3250.0,0.0,1.0
2,1.0,16.0,4000.0,35.0,1.0
3,2.0,0.0,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0
...,...,...,...,...,...
743,0.0,2.0,500.0,38.0,0.0
744,21.0,2.0,500.0,52.0,0.0
745,23.0,0.0,750.0,62.0,0.0
746,39.0,1.0,250.0,39.0,0.0


In [13]:
euc_dist = {}

In [14]:
for i in range(df2.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,5)
dummy

Kolom : 1
[1.0, 2.0, 2.0, 4.0, 12.0]
min :  1.0
max :  12.0
mean :  4.2
fuzzy :  [(12.0, 0.0), (2.0, 0.3125), (4.0, 0.9375), (1.0, 0.0), (2.0, 0.3125)]
fuzzy :  None
pembilang :  5.0
penyebut :  1.5625
rata-rata tertimbang :  3.2
 
[2.0, 2.0, 2.0, 2.0, 4.0]
median :  3
min :  2.0
max :  4.0
mean :  2.4
fuzzy :  [(2.0, 2.0), (2.0, 2.0), (2.0, 2.0), (2.0, 2.0), (4.0, 2.0)]
fuzzy :  2.0
pembilang :  24.0
penyebut :  10.0
rata-rata tertimbang :  2.0
 
[2.0, 7.0, 8.0, 12.0, 23.0]
min :  2.0
max :  23.0
mean :  10.4
fuzzy :  [(23.0, 0.0), (2.0, 0.0), (8.0, 0.7142857142857143), (12.0, 0.873015873015873), (7.0, 0.5952380952380952)]
fuzzy :  None
pembilang :  20.357142857142858
penyebut :  2.182539682539683
rata-rata tertimbang :  9.327272727272726
 
[2.0, 2.0, 4.0, 4.0, 4.0]
median :  3
min :  2.0
max :  4.0
mean :  3.2
fuzzy :  [(4.0, 4.0), (2.0, 4.0), (2.0, 4.0), (4.0, 4.0), (4.0, 4.0)]
fuzzy :  4.0
pembilang :  64.0
penyebut :  20.0
rata-rata tertimbang :  4.0
 
[0.0, 1.0, 2.0, 4.0, 4.0]
mi

[0.0, 2.0, 2.0, 2.0, 16.0]
min :  0.0
max :  16.0
mean :  4.4
fuzzy :  [(0.0, 0.0), (16.0, 0.0), (2.0, 0.45454545454545453), (2.0, 0.45454545454545453), (2.0, 0.45454545454545453)]
fuzzy :  None
pembilang :  2.727272727272727
penyebut :  1.3636363636363635
rata-rata tertimbang :  2.0
 
[0.0, 0.0, 0.0, 21.0, 21.0]
median :  3
min :  0.0
max :  21.0
mean :  8.4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (21.0, 0.0), (21.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 21.0, 21.0, 21.0]
median :  3
min :  0.0
max :  21.0
mean :  12.6
fuzzy :  [(0.0, 21.0), (0.0, 21.0), (21.0, 21.0), (21.0, 21.0), (21.0, 21.0)]
fuzzy :  21.0
pembilang :  1323.0
penyebut :  105.0
rata-rata tertimbang :  21.0
 
[0.0, 1.0, 4.0, 5.0, 11.0]
min :  0.0
max :  11.0
mean :  4.2
fuzzy :  [(0.0, 0.0), (1.0, 0.23809523809523808), (5.0, 0.8823529411764706), (4.0, 0.9523809523809523), (11.0, 0.0)]
fuzzy :  None
pembilang :  8.4593837535014
penyebut :  2.072829131652661


[0.0, 7.0, 7.0, 7.0, 7.0]
median :  3
min :  0.0
max :  7.0
mean :  5.6
fuzzy :  [(0.0, 7.0), (7.0, 7.0), (7.0, 7.0), (7.0, 7.0), (7.0, 7.0)]
fuzzy :  7.0
pembilang :  196.0
penyebut :  35.0
rata-rata tertimbang :  7.0
 
[8.0, 8.0, 8.0, 8.0, 8.0]
median :  3
min :  8.0
max :  8.0
mean :  8.0
fuzzy :  [(8.0, 8.0), (8.0, 8.0), (8.0, 8.0), (8.0, 8.0), (8.0, 8.0)]
fuzzy :  8.0
pembilang :  320.0
penyebut :  40.0
rata-rata tertimbang :  8.0
 
[8.0, 8.0, 8.0, 8.0, 8.0]
median :  3
min :  8.0
max :  8.0
mean :  8.0
fuzzy :  [(8.0, 8.0), (8.0, 8.0), (8.0, 8.0), (8.0, 8.0), (8.0, 8.0)]
fuzzy :  8.0
pembilang :  320.0
penyebut :  40.0
rata-rata tertimbang :  8.0
 
[0.0, 3.0, 3.0, 3.0, 3.0]
median :  3
min :  0.0
max :  3.0
mean :  2.4
fuzzy :  [(0.0, 3.0), (3.0, 3.0), (3.0, 3.0), (3.0, 3.0), (3.0, 3.0)]
fuzzy :  3.0
pembilang :  36.0
penyebut :  15.0
rata-rata tertimbang :  3.0
 
[0.0, 7.0, 7.0, 7.0, 7.0]
median :  3
min :  0.0
max :  7.0
mean :  5.6
fuzzy :  [(0.0, 7.0), (7.0, 7.0), (7.0, 7.0),

[500.0, 500.0, 500.0, 750.0, 1000.0]
min :  500.0
max :  1000.0
mean :  650.0
fuzzy :  [(500.0, 0.0), (500.0, 0.0), (500.0, 0.0), (1000.0, 0.0), (750.0, 0.7142857142857143)]
fuzzy :  None
pembilang :  535.7142857142858
penyebut :  0.7142857142857143
rata-rata tertimbang :  750.0000000000001
 
[0.0, 250.0, 250.0, 250.0, 250.0]
median :  3
min :  0.0
max :  250.0
mean :  200.0
fuzzy :  [(0.0, 250.0), (250.0, 250.0), (250.0, 250.0), (250.0, 250.0), (250.0, 250.0)]
fuzzy :  250.0
pembilang :  250000.0
penyebut :  1250.0
rata-rata tertimbang :  250.0
 
[250.0, 250.0, 250.0, 250.0, 250.0]
median :  3
min :  250.0
max :  250.0
mean :  250.0
fuzzy :  [(250.0, 250.0), (250.0, 250.0), (250.0, 250.0), (250.0, 250.0), (250.0, 250.0)]
fuzzy :  250.0
pembilang :  312500.0
penyebut :  1250.0
rata-rata tertimbang :  250.0
 
[250.0, 250.0, 250.0, 250.0, 250.0]
median :  3
min :  250.0
max :  250.0
mean :  250.0
fuzzy :  [(250.0, 250.0), (250.0, 250.0), (250.0, 250.0), (250.0, 250.0), (250.0, 250.0)]
fu

[0.0, 0.0, 250.0, 250.0, 250.0]
median :  3
min :  0.0
max :  250.0
mean :  150.0
fuzzy :  [(0.0, 250.0), (0.0, 250.0), (250.0, 250.0), (250.0, 250.0), (250.0, 250.0)]
fuzzy :  250.0
pembilang :  187500.0
penyebut :  1250.0
rata-rata tertimbang :  250.0
 
[0.0, 5500.0, 10250.0, 10750.0, 11500.0]
min :  0.0
max :  11500.0
mean :  7600.0
fuzzy :  [(0.0, 0.0), (10250.0, 0.32051282051282054), (11500.0, 0.0), (5500.0, 0.7236842105263158), (10750.0, 0.19230769230769232)]
fuzzy :  None
pembilang :  9332.82726045884
penyebut :  1.2365047233468287
rata-rata tertimbang :  7547.74897680764
 
[0.0, 500.0, 500.0, 500.0, 1000.0]
min :  0.0
max :  1000.0
mean :  500.0
fuzzy :  [(0.0, 0.0), (500.0, 1.0), (500.0, 1.0), (500.0, 1.0), (1000.0, 0.0)]
fuzzy :  None
pembilang :  1500.0
penyebut :  3.0
rata-rata tertimbang :  500.0
 
[0.0, 750.0, 750.0, 1250.0, 1250.0]
min :  0.0
max :  1250.0
mean :  800.0
fuzzy :  [(0.0, 0.0), (750.0, 0.9375), (750.0, 0.9375), (1250.0, 0.0), (1250.0, 0.0)]
fuzzy :  None
pe

...

[0.0, 1.0, 1.0, 1.0, 1.0]
median :  3
min :  0.0
max :  1.0
mean :  0.8
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  4.0
penyebut :  5.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0]
median :  3
min :  0.0
max :  1.0
mean :  0.8
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  4.0
penyebut :  5.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0]
median :  3
min :  0.0
max :  1.0
mean :  0.8
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  4.0
penyebut :  5.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0]
median :  3
min :  0.0
max :  1.0
mean :  0.8
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  4.0
penyebut :  5.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0]
median :  3
min :  0.0
max :  1.0
mean :  0.8
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)

[0.0, 1.0, 1.0, 1.0, 1.0]
median :  3
min :  0.0
max :  1.0
mean :  0.8
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  4.0
penyebut :  5.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 0.0, 0.0, 1.0]
median :  3
min :  0.0
max :  1.0
mean :  0.2
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (1.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 1.0, 1.0]
median :  3
min :  0.0
max :  1.0
mean :  0.4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (1.0, 0.0), (1.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 1.0, 1.0, 1.0]
median :  3
min :  0.0
max :  1.0
mean :  0.6
fuzzy :  [(0.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  3.0
penyebut :  5.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0]
median :  3
min :  0.0
max :  1.0
mean :  0.8
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)

[0.0, 1.0, 1.0, 1.0, 1.0]
median :  3
min :  0.0
max :  1.0
mean :  0.8
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  4.0
penyebut :  5.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0]
median :  3
min :  0.0
max :  1.0
mean :  0.8
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  4.0
penyebut :  5.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0]
median :  3
min :  0.0
max :  1.0
mean :  0.8
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  4.0
penyebut :  5.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0]
median :  3
min :  0.0
max :  1.0
mean :  0.8
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  4.0
penyebut :  5.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0]
median :  3
min :  0.0
max :  1.0
mean :  0.8
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)

[0.0, 1.0, 1.0, 1.0, 1.0]
median :  3
min :  0.0
max :  1.0
mean :  0.8
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  4.0
penyebut :  5.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0]
median :  3
min :  0.0
max :  1.0
mean :  0.8
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  4.0
penyebut :  5.0
rata-rata tertimbang :  1.0
 


,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.000000,50.0,12500.0,98.000000,1.0
1,3.200000,13.0,3250.0,47.925107,1.0
2,1.000000,16.0,4000.0,35.000000,1.0
3,2.000000,20.0,5000.0,45.000000,1.0
4,1.000000,24.0,6000.0,77.000000,1.0
...,...,...,...,...,...
743,10.542056,2.0,500.0,38.000000,1.0
744,21.000000,2.0,500.0,52.000000,1.0
745,23.000000,3.0,750.0,62.000000,1.0
746,39.000000,1.0,250.0,39.000000,1.0


In [15]:
dummy.describe()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
count,748.000000,748.000000,748.000000,748.000000,748.0
mean,9.491944,5.516043,1361.925899,33.700896,1.0
std,8.016305,5.839533,1446.575304,23.848776,0.0
min,1.000000,1.000000,250.000000,2.000000,1.0
25%,3.000000,2.000000,500.000000,16.000000,1.0
50%,8.000000,4.000000,1000.000000,28.000000,1.0
75%,14.000000,7.000000,1750.000000,48.000000,1.0
max,74.000000,50.000000,12500.000000,98.000000,1.0


# Klasifikasi

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [17]:
x = dummy.values
y = df.iloc[:, -1].values

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=72)

In [19]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [20]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[130  16]
 [ 28  13]]
0.7647058823529411
